In [2]:
import pandas as pd
import os
import string
import re
from nltk.stem import WordNetLemmatizer
import nltk
import pickle
print('NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)')

print('UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED')
#nltk.download()
stopword = nltk.corpus.stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()

NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)
UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED


In [3]:
def preprocess(df):
    
    df = df[df.columns.drop(list(df.filter(regex='^Cat')))]
    df = df[df['Date'] != '27/06/2001']  #removing the date
    df = df[(df['Subject'] != 'RE:') & (df['Subject'] != 'FW:') & (df['Subject'] != 'Re:')]  #removing the max same subjects
    del df['Unnamed: 0']
    return df

In [4]:
stopword.append('re')
stopword.append('fw')

In [5]:
def clean_text(text):
    text_nopunct = "".join([char for char in text.lower() if char not in string.punctuation])
    tokens = re.split('\W+', text_nopunct) #tokenize
    words_without_stopwords = [word for word in tokens if word not in stopword] #remove stopwords from tokens
    return [wordnet_lemmatizer.lemmatize(word, pos="v") for word in words_without_stopwords]

In [6]:
DATA = 'data' #https://data.world/brianray/enron-email-dataset

FILENAMES = [os.path.join(DATA, filename) for filename in os.listdir(DATA)]
df = pd.read_csv(FILENAMES[0])
df = preprocess(df)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.shape

(96906, 15)

In [8]:
df[df['Subject'].isna()].shape

(5364, 15)

In [9]:
df[df['content'].isna()].shape

(402, 15)

##### Right now just removing the nan value, may look into this later for do preprocessing

In [10]:
df = df[df['Subject'].notna()]
df = df[df['content'].notna()]
df[df['content'].isna()].shape

(0, 15)

In [11]:

dfEmail = df[['Subject', 'content']]
# dfEmail['content_lemma'] = dfEmail['content'].apply(lambda x: clean_text(x))
# dfEmail['Subject_lemma'] = dfEmail['content'].apply(lambda x: clean_text(x))

In [12]:
dfEmail.head()

Subject  \
2                        Re: test   
4                       Re: Hello   
5                       Re: Hello   
7    Re: PRC review - phone calls   
8  Re: High Speed Internet Access   

                                             content  
2                      test successful. way to go!!!  
4                  Let's shoot for Tuesday at 11:45.  
5  Greg, How about either next Tuesday or Thursda...  
7                   any morning between 10 and 11:30  
8  1. login: pallen pw: ke9davis I don't think th...

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer(stop_words='english', analyzer=clean_text)

In [15]:
vectorizer.fit(dfEmail['content'])
print("\n Feature names Identified :\n")
print(vectorizer.get_feature_names())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
print(len(vectorizer.get_feature_names()))

302523


In [18]:
#saveing content vectorizer for future use
pickle.dump(vectorizer, open("tfidf_content.pickle", "wb"))

simillarly, we will do the same on subject

In [19]:
vectorizer_subject = TfidfVectorizer(stop_words='english', analyzer=clean_text)
vectorizer_subject.fit(dfEmail['Subject'])
print("\n Feature names Identified :\n")
print(vectorizer_subject.get_feature_names())
print(len(vectorizer_subject.get_feature_names()))
#saveing content vectorizer for future use
pickle.dump(vectorizer_subject, open("tfidf_subject.pickle", "wb"))


 Feature names Identified :

['', '0', '00', '000', '00000000000000000000000000000000000weil', '000000000055936', '000000000078904', '0000105674', '0000108727', '0000110257', '0000110295', '0000110326', '0000110685', '0000264227478000012', '0002001', '0008002', '0008037', '001', '0010002', '0010028', '0010038', '001018xls', '0011001', '00110100095649', '0011038', '0011056', '00240833807905653', '00262869908194660', '003', '0048', '0060', '0074', '00ceovol1', '01', '010', '0100', '010000004', '010001522', '010102', '010202', '010402', '010501', '0105064', '01070111', '010702', '01074', '01082002', '010902', '01092002', '011101', '011115', '01140118', '011402', '01142002', '011442076824251', '0115', '01152002', '01154', '01155', '01158', '011602', '01162', '01162002', '01164', '011702', '01172', '01172002', '01174', '01176', '01177', '01178', '0118', '01181', '01182002', '01186', '01187', '01188', '011901', '01210125', '012201', '012202', '012301', '01235', '01236', '01237', '01238pdf',

In [20]:
data = vectorizer.transform(['Traveling to have a business meeting takes th'])

In [21]:
print(data)

  (0, 279612)	0.5635233351271743
  (0, 273885)	0.5925793180347765
  (0, 270437)	0.3191881310702937
  (0, 196672)	0.31236505526153147
  (0, 87943)	0.36309532059763416


In [22]:
print(len(list(data.toarray()[0])))

302523


In [23]:
data = vectorizer_subject.transform(['Traveling to have a business meeting takes th'])
print(data)

  (0, 16520)	0.46900268602511774
  (0, 16199)	0.5978242881136456
  (0, 15960)	0.46170601571821973
  (0, 10932)	0.24616210923295878
  (0, 4164)	0.3858424187025391


In [24]:
print(len(list(data.toarray()[0])))

18001


In [25]:
import numpy as np

In [26]:
np.array(data.toarray()[0]).shape

(18001,)

In [27]:
np.array([np.array(data.toarray()[0]).shape, np.array(data.toarray()[0]).shape]).shape

(2, 1)